# UDACITY SageMaker Essentials: Training Job Exercise

Good job on your work so far! You've gotten an overview of building an ML Workflow in AWS. Now, it's time to practice your skills. In this exercise, you will be training a BlazingText model to help predict the helpfulness of Amazon reviews. The model & parameters have already been chosen for you; it's your task to properly upload the data necessary for the job and launch the training.  

In [2]:
import os
import boto3
import sagemaker
import json
import zipfile

import pandas as pd
import numpy as np



## Preprocessing

The data we'll be examining today is a collection of reviews for an assortment of toys and games found on Amazon. This data includes, but is not limited to, the text of the review itself as well as the number of user "votes" on whether or not the review was helpful. Today, we will be making a model that predicts the usefulness of a review, given only the text of the review. This is an example of a problem in the domain of supervised sentiment analysis; we are trying to extract something subjective from text given prior labeled text.

Before we get started, we want to know what form of data is accepted in the algorithm we're using. We'll be using BlazingText, an implemention of Word2Vec optimized for SageMaker. In order for this optimization to be effective, data needs to be preprocessed to match the correct format. The documentation for this algorithm can be found here: https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html

We will be training under "File Mode", which requires us to do two things in preprocessing this data. First, we need to generate labels from the votes. For this exercise, if the majority of votes for a review is helpful, we will designate it \_\_label\_\_1, and if the majority of votes for a review is unhelpful, we will designate it \_\_label\_\_2. In the edge case where the values are equal, we will drop the review from consideration. Second, we need to separate the sentences, while keeping the original label for the review. These reviews will often consist of several sentences, and this algorithm is optimized to perform best on many small sentences rather than fewer larger paragraphs. We will separate these sentences by the character "."


(This process is obviously very naive, but we will get remarkable results even without a lot of finetuning!)

This preprocessing is done for you in the cells below. Make sure you go through the code and understand what's being done in each step. 

In [3]:
import zipfile

# Function below unzips the archive to the local directory. 

def unzip_data(input_data_path):
    with zipfile.ZipFile(input_data_path, 'r') as input_data_zip:
        input_data_zip.extractall('.')

# Input data is a file with a single JSON object per line with the following format: 
# {
#  "reviewerID": <string>,
#  "asin": <string>,
#  "reviewerName" <string>,
#  "helpful": [
#    <int>, (indicating number of "helpful votes")
#    <int>  (indicating total number of votes)
#  ],
#  "reviewText": "<string>",
#  "overall": <int>,
#  "summary": "<string>",
#  "unixReviewTime": <int>,
#  "reviewTime": "<string>"
# }
# 
# We are specifically interested in the fields "helpful" and "reviewText"
#

def label_data(input_data):
    labeled_data = []
    HELPFUL_LABEL = "__label__1"
    UNHELPFUL_LABEL = "__label__2"
     
    for l in open(input_data, 'r'):
        l_object = json.loads(l)
        helpful_votes = float(l_object['helpful'][0])
        total_votes = l_object['helpful'][1]
        reviewText = l_object['reviewText']
        if total_votes != 0:
            if helpful_votes / total_votes > .5:
                labeled_data.append(" ".join([HELPFUL_LABEL, reviewText]))
            elif helpful_votes / total_votes < .5:
                labeled_data.append(" ".join([UNHELPFUL_LABEL, reviewText]))
          
    return labeled_data


# Labeled data is a list of sentences, starting with the label defined in label_data. 

def split_sentences(labeled_data):
    split_sentences = []
    for d in labeled_data:
        label = d.split()[0]        
        sentences = " ".join(d.split()[1:]).split(".") # Initially split to separate label, then separate sentences
        for s in sentences:
            if s: # Make sure sentences isn't empty. Common w/ "..."
                split_sentences.append(" ".join([label, s]))
    return split_sentences


input_data  = unzip_data('Toys_and_Games_5.json.zip')
labeled_data = label_data('Toys_and_Games_5.json')
split_sentence_data = split_sentences(labeled_data)

print(split_sentence_data[0:9])


['__label__1 Love the magnet easel', '__label__1  great for moving to different areas', '__label__1  Wish it had some sort of non skid pad on bottom though', '__label__1 Both sides are magnetic', "__label__1  A real plus when you're entertaining more than one child", '__label__1  The four-year old can find the letters for the words, while the two-year old can find the pictures the words spell', '__label__1  (I bought letters and magnetic pictures to go with this board)', '__label__1  Both grandkids liked it a lot, which means I like it a lot as well', '__label__1  Have not even introduced markers, as this will be used strictly as a magnetic board']


## Exercise: Upload Data

Your first responsibility is to separate that `split_sentence_data` into a `training_file` and a `validation_file`. Have the training file make up 90% of the data, and have the validation file make up 10% of the data. Careful that the data doesn't overlap! (This will result in overfitting, which might result in nice validation metrics, but crummy generalization.)

Using the methodology of your choice, upload these files to S3. (In practice, it's important to know how to do this through the console, programatically, and through the CLI. If you're feeling frisky, try all 3!) If you're doing this programatically, the Boto3 documentation would be a good reference. https://boto3.amazonaws.com/v1/documentation/api/latest/index.html

The BUCKET will be the name of the bucket you wish to upload it to. The s3_prefix will be the name of the desired 'file-path' that you upload your file to within the bucket. For example, if you wanted to upload a file to:

"s3://example-bucket/1/2/3/example.txt

The "BUCKET" will be 'example-bucket', and the s3_prefix would be '1/2/3'

The code below shows you how to upload it programatically.

In [9]:
import boto3
from botocore.exceptions import ClientError
# Note: This section implies that the bucket below has already been made and that you have access
# to that bucket. You would need to change the bucket below to a bucket that you have write
# premissions to. This will take time depending on your internet connection, the training file is ~ 40 mb

BUCKET = "CHANGE THIS"
s3_prefix = "CHANGE THIS"


def cycle_data(fp, data):
    for d in data:
        fp.write(d + "\n")

def write_trainfile(split_sentence_data):
    train_path = "hello_blaze_train"
    with open(train_path, 'w') as f:
        cycle_data(f, split_sentence_data)
    return train_path

def write_validationfile(split_sentence_data):
    validation_path = "hello_blaze_validation"
    with open(validation_path, 'w') as f:
        cycle_data(f, split_sentence_data)
    return validation_path 

def upload_file_to_s3(file_name, s3_prefix):
    object_name = os.path.join(s3_prefix, file_name)
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, BUCKET, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    
# Split the data
split_data_trainlen = int(len(split_sentence_data) * .9)
split_data_validationlen = int(len(split_sentence_data) * .1)

# Todo: write the training file
train_path = write_trainfile()
print("Training file written!")

# Todo: write the validation file
validation_path = write_validationfile()
print("Validation file written!")

upload_file_to_s3(train_path, s3_prefix)
print("Train file uploaded!")
upload_file_to_s3(validation_path, s3_prefix)
print("Validation file uploaded!")

print(" ".join([train_path, validation_path]))

## Exercise: Train SageMaker Model

Believe it or not, you're already almost done! Part of the appeal of SageMaker is that AWS has already done the heavy implementation lifting for you. Launch a "BlazingText" training job from the SageMaker console. You can do so by searching "SageMaker", and navigating to Training Jobs on the left hand side. After selecting "Create Training Job", perform the following steps:
* Select "BlazingText" from the algorithms available. 
* Specify the "file" input mode of training. 
* Under "resource configuration", select the "ml.m5.large" instance type. Specify 5 additional GBs of memory. 
* Set a stopping condition for 15 minutes. 
* Under hyperparameters, set "mode" to "supervised"
* Under input_data configuration, input the S3 path to your training and validation datasets under the "train" and "validation" channels. You will need to create a channel named "validation".  
* Specify an output path in the same bucket that you uploaded training and validation data. 


If successful, you should see a training job launch in the UI. Go grab a coffee, this will take a little bit of time. If there was a failure, you should see it there. Googling the error should direct 

## Citations

Ups and downs: Modeling the visual evolution of fashion trends with one-class collaborative filtering  
R. He, J. McAuley  
WWW, 2016


Image-based recommendations on styles and substitutes  
J. McAuley, C. Targett, J. Shi, A. van den Hengel  
SIGIR, 2015
